# Imports nécessaires

In [36]:
import pandas as pd
from datetime import datetime

# Classe qui a pour but de retourner efficacement toutes les destinations d'une ville

In [57]:
SEUIL_DISTANCE = 0.03

class DestiOptis:
    stops = []
    calendar_dates = []
    routes = []
    stop_times = []
    trips = []
    lat = 0
    lon = 0
    date_min = None
    date_max = None

    def __init__(self, path ='TER'):
        self.calendar_dates = pd.read_csv('Data/'+ path +'/calendar_dates.txt')
        self.stop_times = pd.read_csv('Data/'+ path +'/stop_times.txt')[['trip_id', 'stop_id', 'departure_time']]
        self.stops = pd.read_csv('Data/'+ path +'/stops.txt')[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'parent_station']]
        self.trips = pd.read_csv('Data/'+ path +'/trips.txt')[['service_id', 'trip_id', 'route_id']]
        self.calendar_dates.date = pd.to_datetime(self.calendar_dates["date"], format='%Y%m%d')
        self.stop_times['departure_time'] = pd.to_timedelta(self.stop_times['departure_time'])
        self.stops['parent_station'] = self.stops['parent_station'].fillna('')
    
    def load_search_destinations(self, lat, lon ,date_min= datetime.today,date_max = datetime.today):
        self.lat = lat
        self.lon = lon
        self.date_min = datetime.strptime(date_min, '%Y%m%d')
        self.date_max = datetime.strptime(date_max, '%Y%m%d')
    
    # Etape 1 : Récupérer les stops de la ville / StopArea
    def get_stops(self):
        self.id_ville = self.villes_proches(self.lat, self.lon)
        ville_Set = set(self.id_ville['stop_id'].array)
        return self.stops[[set([l]).issubset(ville_Set) for l in self.stops.parent_station.values.tolist()]]['stop_id']

    # Retourne les stopArea proche du point de départ
    def villes_proches(self,lat,long):
        return self.stops[(self.stops['stop_lat'] > lat-SEUIL_DISTANCE) & (self.stops['stop_lat'] < lat+SEUIL_DISTANCE) & (self.stops['stop_lon'] > long-SEUIL_DISTANCE) & (self.stops['stop_lon'] < long+SEUIL_DISTANCE) & self.stops['stop_id'].str.contains('StopArea')]
    

In [60]:
# Initialisation pour les tests
Desti = DestiOptis()

In [48]:
print(Desti.calendar_dates.head())
print(Desti.calendar_dates.date[0]+Desti.stop_times.departure_time[0])

   service_id       date  exception_type
0           1 2024-06-30               1
1           1 2024-07-07               1
2           1 2024-07-14               1
3           1 2024-07-21               1
4           1 2024-07-28               1
2024-06-30 07:28:00
